### What is the model-complexity-reducer (mcr) and what is it for?

The mcr tool is designed to maximize the tradeoff between the complexity of GBT algorithms and their performance. In this context, by complexity, we refer to the number of operations the algorithm requires to perform an inference (number of trees and their depth).

This tool is specifically designed so that the resulting model is manageable within the ZKML paradigm, but its functionality can be extended to any problem where:

- The model's weight needs to be minimal, for example for mobile applications.
- Minimal inference times are required for low latency applications.
- We want to check if we have created an overly complex model and a simpler one would give us the same performance (or even better).
- The number of steps required to perform the inference must be less than X (as is currently constrained by the ZKML paradigm).

Now that we understand the possible applications of this tool, let's see how it works.

### How does mcr work?

The best way to understand how our dimensionality reduction algorithm works is to explain step by step the operations it performs:

- Given a model, it retrieves the type of model it is and its important parameters.
- Based on these parameters, it adjusts the search space so that the architectures it will try to find are lighter than the original.
- It carries out a Bayesian optimization process, where it adds as possible hyperparameters:
    - Model-specific parameters adjusted to the new search space.
    - Transformations of the input data using different dimensionality reduction techniques.
- After obtaining the best parameters for a given evaluation metric, it returns:
    - A transformer with the transformation that must be applied to the test set to generate the inference.
    - The newly adjusted model.

Within the world of ZKML, we know that the cost of executing a ZK Proof is much higher than the cost of generating a normal inference, due to the cryptographic process involved.
In this context, mcr has been developed with the goal of creating models that can be transpiled into others. These new models can be interpreted by other programs (in our case, Cairo) and generate ZK Proofs.

In [ ]:
# For this example, it is necessary to have both xgboost and lightgbm installed, but it is not necessary to have all packages installed to use auto_zkml. 
# For this reason, we include this cell to ensure the notebook works correctly.

!pip install xgboost
!pip install lightgbm


In this simple tutorial, we will only look at how to use the package, without delving into performance details. To see these details, please refer to the notebook end_to_end_using_giza_stack.ipynb.

In [ ]:
import lightgbm as lgbm
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

data = load_diabetes()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

n_estimators = 1200
max_depth = 8

lgbm_reg = lgbm.LGBMRegressor(n_estimators=n_estimators, max_depth=max_depth)
lgbm_reg.fit(X, y)

In [2]:
lgbm_reg.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': 8,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 1200,
 'n_jobs': None,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [3]:
from auto_zkml import mcr

In [ ]:
model, transformer = mcr(model = lgbm_reg,
                         X_train = X_train,
                         y_train = y_train, 
                         X_eval = X_test, 
                         y_eval = y_test, 
                         eval_metric = 'rmse', 
                         transform_features = True)

In [5]:
model.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': 3,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 200,
 'n_jobs': None,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'min_data_in_leaf': 72,
 'feature_fraction': 0.16836383405441807,
 'bagging_fraction': 0.24326086678781003,
 'verbose': -1,
 'early_stopping_rounds': 10}

As we can observe, the complexity of the model has drastically decreased. We have gone from 1200 trees of depth 8 to 200 trees of depth 3. In the notebook end_to_end_example.ipynb, we will see the performance of our new models after reducing their complexity.







We now have our model and our transformer ready for making inferences!